# Convergence criteria
In this tutorial we will run a multiphysics simulation consisting of fully coupled single phase flow and mechanics. We present different ways of using convergence criteria. We differentiate between utilizing Euclidean norms for the full algebraic vectors occuring during iteration and Lebesgue norms that take into account the single sub-physics. In addition, we compare absolute and relative criteria.

The examples here are based on those in the [Poromechanics tutorial](./poromechanics.ipynb). Yet, we use looser tolerances to better dive into the differences of the convergence criteria.

# Poromechanics model

We redefine the same model as in the poromechanics tutorial. See [poromechanics tutorial](./poromechanics.ipynb) for explanations. The only difference is the use of slightly different material parameters inducing a more notable disparity between the scaling in the displacement and pressure variables.

In [ ]:
from porepy.applications.md_grids.model_geometries import (
    SquareDomainOrthogonalFractures,
)
import porepy as pp
import numpy as np

In [ ]:
class BodyForceMixin:
    nd: int
    """Ambient dimension."""

    units: pp.Units

    solid: pp.SolidConstants

    def body_force(self, subdomains: list[pp.Grid]) -> pp.ad.Operator:
        units = self.units
        vals = []
        for sd in subdomains:
            data = np.zeros((sd.num_cells, self.nd))

            # We add the source only to the 2D domain and not the fracture.
            if sd.dim == 2:
                # Selecting central cells
                cell_centers = sd.cell_centers
                indices = (
                    (cell_centers[0] > (0.3 / units.m))
                    & (cell_centers[0] < (0.7 / units.m))
                    & (cell_centers[1] > (0.3 / units.m))
                    & (cell_centers[1] < (0.7 / units.m))
                )

                acceleration = self.units.convert_units(-9.8, "m * s^-2")
                force = self.solid.density * acceleration
                data[indices, 1] = force * sd.cell_volumes[indices]

            vals.append(data)
        return pp.ad.DenseArray(np.concatenate(vals).ravel(), "body_force")

In [ ]:
class PressureSourceBC:
    def bc_type_darcy_flux(self, sd: pp.Grid) -> pp.BoundaryCondition:
        """Assign Dirichlet boundary condition to the north boundary and Neumann
        everywhere else.

        """
        domain_sides = self.domain_boundary_sides(sd)
        bc = pp.BoundaryCondition(sd, domain_sides.north, "dir")
        return bc

    def fluid_source(self, subdomains: list[pp.Grid]) -> pp.ad.Operator:
        """Assign fracture source."""
        # Retrieve internal sources (jump in mortar fluxes) from the base class
        internal_sources: pp.ad.Operator = super().fluid_source(subdomains)

        # Retrieve external (integrated) sources from the exact solution.
        values = []
        src_value: float = self.units.convert_units(0.1, "kg * m^-3 * s^-1")
        for sd in subdomains:
            if sd.dim == self.mdg.dim_max():
                values.append(np.zeros(sd.num_cells))
            else:
                values.append(np.ones(sd.num_cells) * src_value)

        external_sources = pp.wrap_as_dense_ad_array(np.concatenate(values))

        # Add up both contributions
        source = internal_sources + external_sources
        source.set_name("fluid sources")

        return source

We define an additional mixin, which allows to log norms over the course of the nonlinear iterations.

In [ ]:
from porepy.viz.solver_statistics import NonlinearSolverStatistics

class CustomStatisticsLogger:
    """Logs custom norms during the nonlinear iterations."""

    nonlinear_solver_statistics: NonlinearSolverStatistics

    def after_nonlinear_iteration(self, nonlinear_increment: np.ndarray):
        increment_norms = self.variable_norm(nonlinear_increment)
        residual = self.equation_system.assemble(evaluate_jacobian=False)
        residual_norms = self.residual_norm(residual)

        self.nonlinear_solver_statistics.log_custom_data(
            append=True,
            **{"increment_norms": increment_norms,"residual_norms": residual_norms},
        )
        super().after_nonlinear_iteration(nonlinear_increment)

We redefine the poromechanics model, called PoromechanicsSourceBC in the poromechanics tutorial, and add the additional logging.

In [ ]:
class PlainModel(
    CustomStatisticsLogger,
    PressureSourceBC,
    BodyForceMixin,
    SquareDomainOrthogonalFractures,
    pp.Poromechanics,
):
    """Adding geometry, boundary conditions and source to the default model."""

    def meshing_arguments(self) -> dict:
        cell_size = self.units.convert_units(0.1, "m")
        return {"cell_size": cell_size}

In addition, we define the same model, which however uses Lebesgue-type multiphysics norms.

In [ ]:
from porepy.models.metric import MultiphysicsEuclideanMetric
class ModelWithMultiphysicsNorms(
    MultiphysicsEuclideanMetric,
    PlainModel,
):
    """Adding multiphysics norms to the custom poromechanics model."""
    ...

Now we define instances of the two model classes.

In [ ]:
plain_model = PlainModel()
multi_norm_model = ModelWithMultiphysicsNorms()

We define convergence criteria and tolerances, as part of the solver parameters. The default case uses absolute convergence criteria. Let's on purpose use some looser tolerances.

In [ ]:
from porepy.numerics.nonlinear.convergence_check import ConvergenceTolerance, DynamicRelativeConvergenceCriterion

solver_params = {
    "nl_convergence_tol": ConvergenceTolerance(
        tol_increment = 1e-2, # Controls the tolerance of the norm of the separate increments
        tol_residual= 1e-2, # Controls the tolerance of the norm of the separate residuals
    ),
}

We are ready to run the models with different options.

In [ ]:
pp.run_time_dependent_model(plain_model, solver_params)
pp.run_time_dependent_model(multi_norm_model, solver_params)

We compare the solutions in the eye-norm.

In [ ]:
pp.plot_grid(
    plain_model.mdg,
    cell_value=plain_model.pressure_variable,
    vector_value=plain_model.displacement_variable,
    figsize=(10, 8),
    title="Pressure and displacement (singlephysics norm)",
)

pp.plot_grid(
    multi_norm_model.mdg,
    cell_value=multi_norm_model.pressure_variable,
    vector_value=multi_norm_model.displacement_variable,
    figsize=(10, 8),
    title="Pressure and displacement (multiphysics norm)",
)

We compare the number of nonlinear iterations.

In [ ]:
# Compare the number of iterations for the two models
print(f"Number of iterations (plain model): {plain_model.nonlinear_solver_statistics.num_iteration}")
print(f"Number of iterations (multi-physics  criteria): {multi_norm_model.nonlinear_solver_statistics.num_iteration}")

Let's dive into the logged custom data of the multi norm model. Essentially the two models are the same, the latter has simply gone a single iteration further.

In [ ]:
import pprint
pprint.pprint(multi_norm_model.nonlinear_solver_statistics.custom_data, width=80, depth=3)

To conclude, in the above comparison, the single-physics criteria do not see a great difference in continuing for two more nonlinear iterations, whereas the multi-physics norms are sensitive to the single components and request further iteration. 

Diving a bit further into the details, it is mostly interface quantities which do not get seen by a global apporach, as they get lumped together with the rest, and if much fewer interface cells are present, these get overseen by the global Euclidean norm. By splitting, and checking the convergence on each, we typically get better control on the convergence of the different sub-physics.

## Lebesgue norms (in constrast to Euclidean norms)
The present example has a regular grid. Thus, there is no strong need to take into account the cell sizes as a weight in the norm. This can drastically change, when considering multi-fractured domains with tiny cells close to multiple fractures intersecting. Then, the Euclidean norms, which do not see any scaling are not sensitive to teh size of the cells and naturally count in tiny cells in the same degree as large cells. Also in grid refinement studies (although the Euclidean norms in PorePy are scaled by the overall size of the algebraic problem), the use of cell-weighted norms is size-invariant. 

In order to activate Lebesgue norms, we use mixins.

In [ ]:
from porepy.models.metric import MultiphysicsLebesgueMetric

class ModelWithMultiphysicsLebesgueNorms(
    MultiphysicsLebesgueMetric,
    PlainModel,
):
    """Adding multiphysics Lebesgue norms to the custom poromechanics model."""
    ...

multi_lebesgue_model = ModelWithMultiphysicsLebesgueNorms()
pp.run_time_dependent_model(multi_lebesgue_model, solver_params)
print(f"Number of iterations (multi-physics Lebesgue criteria): {multi_lebesgue_model.nonlinear_solver_statistics.num_iteration}")

## Relative convergence criteria
In some situations (not here), it is necessary to use relative criteria. Such can be activated by providing additional solver parameters. We skew the units to make the point.

In [ ]:
params = {
    "units": pp.Units(m=1, kg=1e-4)
}

For a direct comparison, we define the same model and run with two different solver parameter sets.

In [ ]:
from porepy.numerics.nonlinear.convergence_check import DynamicRelativeConvergenceCriterion

solver_params_with_relative_criteria = {
    "nl_convergence_tol": ConvergenceTolerance(
        tol_increment = 1e-2, # Controls the tolerance of the norm of the separate increments
        tol_residual= 1e-2, # Controls the tolerance of the norm of the separate residuals
    ),
    "nl_convergence_criterion": DynamicRelativeConvergenceCriterion(),
}

multi_norm_model_with_skewed_units = ModelWithMultiphysicsNorms(params)
multi_norm_model_with_skewed_units_and_relative_criteria = ModelWithMultiphysicsNorms(params)

pp.run_time_dependent_model(multi_norm_model_with_skewed_units, solver_params)
pp.run_time_dependent_model(multi_norm_model_with_skewed_units_and_relative_criteria, solver_params_with_relative_criteria)

print(f"Number of iterations (restarted model): {multi_norm_model_with_skewed_units.nonlinear_solver_statistics.num_iteration}")
print(f"Number of iterations (restarted model with relative criteria): {multi_norm_model_with_skewed_units_and_relative_criteria.nonlinear_solver_statistics.num_iteration}")

We see that the absolute criteria required an extra iteration here. Let's checkout the history of the residual norms. Indeed, the absolute criteria track a relative decrease of about 1e-6, but since the `kg` unit is scaled, some of the equations are not scaled on the order of 1 (as for the uniform parameters above).

In [ ]:
print("Residual history for absolute criteria:")
pprint.pprint(multi_norm_model_with_skewed_units.nonlinear_solver_statistics.residual_norms, width=80, depth=3)
print("\nResidual history for relative criteria:")
pprint.pprint(multi_norm_model_with_skewed_units_and_relative_criteria.nonlinear_solver_statistics.residual_norms, width=80, depth=3)

As a final deep-dive, lets double check the different subphysics norms over the different iterations. These are measured the same for both models. Thus, we only look at the first model. Indeed, we can see several quantities like the pressure as well as the momentum equation being scaled away from order one.

In [ ]:
pprint.pprint(multi_norm_model_with_skewed_units.nonlinear_solver_statistics.custom_data, width=80, depth=3)

## Solver statistics
As a bonus, let us briefly present the overall data logged by the solver statistics object. As the problem is time dependent, and nonlinear, relevant parameters are tracked.

In [ ]:
import pprint
for key, value in restarted_multi_norm_model.nonlinear_solver_statistics.__dict__.items():
    print(f"Key: {key}")
    pprint.pprint(value, width=80, depth=3)
    print()  # Add blank line between entries

# What we have explored
We se  bit deeper into the convergence behavior of teh poromechanics model, already presented in the [poromechanics tutorial](./poromechanics.ipynb). We explored the possibilities to define different norms for measuring increments and residuals, as well as absolute and relative criteria.